In [1]:
import tifffile as tf 
import zarr

# read tiff file
channel_io = tf.TiffFile('/Users/swarchol/Downloads/exemplar-001 (1)/registration/exemplar-001.ome.tif', is_ome=False)
channels = zarr.open(channel_io.series[0].aszarr())
channels[2]

<zarr.core.Array '/2' (12, 785, 628) uint16>

In [7]:
channels[0][1][2][3][4]

IndexError: invalid index to scalar variable.

In [27]:
# numpy to csv
import numpy as np
values = np.array(channels[2][0].flatten(), dtype=np.uint16)
np.savetxt("values.csv", values,  fmt='%i', delimiter=",")

In [26]:
values

array([59, 76, 72, ..., 65, 68,  0], dtype=uint16)

In [38]:
# read csv to numpy array
import numpy as np
values = np.genfromtxt('./values.csv', delimiter=',')
values

array([59., 76., 72., ..., 65., 68.,  0.])

116.98012752379897 151.56003400292758 14.562541860581288 15.392261187267765
133.91722457480074 182.3445563774631


/var/folders/ps/x7v31pds3gn4t11djwg0t03h0000gp/T/ipykernel_23399/3943088519.py:9: RuntimeWarning: divide by zero encountered in log
  img_log = np.log([image_data > 0])


In [72]:
values.shape

(492980,)

In [74]:
packet_gmm = {}
image_data = values.astype(np.float32)
img_log = np.log(image_data[image_data > 0])
gmm = GaussianMixture(3, max_iter=1000, tol=1e-6)
gmm.fit(img_log.reshape((-1, 1)))

means = gmm.means_[:, 0]
i0, i1, i2 = np.argsort(means)
mean1, mean2 = means[[i1, i2]]
std1, std2 = gmm.covariances_[[i1, i2], 0, 0] ** 0.5

x = np.linspace(mean1, mean2, 50)
y1 = norm(mean1, std1).pdf(x) * gmm.weights_[i1]
y2 = norm(mean2, std2).pdf(x) * gmm.weights_[i2]

lmax = mean2 + 2 * std2
lmin = x[np.argmin(np.abs(y1 - y2))]
if lmin >= mean2:
    lmin = mean2 - 2 * std2
vmin = max(np.exp(lmin), image_data.min(), 0)
vmax = min(np.exp(lmax), image_data.max())

packet_gmm['vmin'] = np.rint(vmin)
packet_gmm['vmax'] = np.rint(vmax)
vmin, vmax

(4839.378206717191, 33805.43652048477)